## ⚙️ 1. Setup: Install Libraries

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Define Workflow Tools

For our iterative workflow, we need two key functions: sum_costs for our accountant agent to do math, and exit_loop for our refiner agent to signal when the goal is met and the loop should terminate.

## 4. Create Tool Wrappers

To ensure our cost_cutter_agent can reliably use both the built-in Google Search and our custom exit_loop function, we'll wrap google search agent as a tool agent

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool

google_search_agent = Agent(
    name="Google_Search_agent",
    model="gemini-2.5-flash",
    instruction="You are just a wrapper for the Google Search tool.",
    tools=[google_search]
)

google_search_tool = AgentTool(agent=google_search_agent)

## 📝 5. Create the Proposer, Accountant, Cost Cutter and Plan Retriever Agents

###Proposer:
This is the first agent in our workflow. It runs only once to propose an initial plan based on the user's topic. Its output is stored in the current_plan variable.

###Accountant
This agent that runs at the beginning of each loop. It takes the current_plan, checks it against the budget using its sum_costs tool, and outputs a critique.

###Cost Cutter
This agent is the second step in our loop. It takes the critique as input. If the budget is not met, it uses its Google Search Tool to find a cheaper option. If the budget is met, it calls the exit_loop tool to terminate the process.

###Plan Retriever
This agent's only job is to run after the loop has successfully finished. It takes the final current_plan from the session state and presents it clearly to the user.

## 🔄 6. Assemble the Loop and Sequential Workflows

Now we assemble our agent team. The LoopAgent is created to handle the critique-and-refine cycle. Then, a top-level SequentialAgent puts the entire "propose -> loop -> present" workflow together.

## 🚀 7. Build the Execution Engine

This is our helper function for running queries, unchanged from previous articles. It handles the core ADK logic of initializing the Runner and streaming events.

In [ ]:
from IPython.display import display, Markdown

from google.adk.sessions import Session
from google.genai.types import Content, Part
from google.adk.runners import Runner

async def run_agent_query(agent: Agent, query: str, topic: str, budget: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user"),
            state_delta={'budget': budget, 'topic': topic} # Pass budget and topic to context
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response


## ✨ 8. Initialize Session and Run the Workflow

Finally, we set up our InMemorySessionService and define our main execution block. We set the initial budget and topic in the session state, then kick off the entire autonomous workflow with a single call.

In [ ]:
from google.adk.sessions import InMemorySessionService


# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [ ]:
async def run_stateful_orchestrator():

  session = await session_service.create_session(
        app_name=budget_optimizer_workflow.name,
        user_id=user_id
  )

  budget=15000
  topic="50 person AI event in New York"
  query = f"Find a plan for {topic}"

  print(f"User: {query}\n")
  await run_agent_query(budget_optimizer_workflow, query, topic, budget, session, user_id)

# Run the full system
await run_stateful_orchestrator()